## Load CSV from spotify

In [1]:
import pandas as pd
import numpy as np

In [15]:
df_spotify = pd.read_csv('SpotifyFeatures.csv')

In [16]:
Genre_to_remove = ['Anime', "Children's Music", 'Children’s Music', 'Classical', 'Soundtrack', 'Opera', 'A Capella', 'Movie']
for genre in Genre_to_remove:
    df_spotify = df_spotify[df_spotify.genre != genre]

In [17]:
to_search = df_spotify[['artist_name', 'track_name', 'popularity', 'genre']]

We need to remove all the occurences of feat

In [18]:
def remove_feat(track_name):
    if '(' in track_name:
        track = track_name.split('(')
        return track[0]
    else:
        return track_name
    
def remove_space(track_name):
    if ' - ' in track_name:
        track = track_name.split(' - ')
        return track[0]
    else:
        return track_name

In [19]:
to_search['track_name'] = to_search['track_name'].apply(lambda x: remove_feat(x))

C:\Users\Alexis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
to_search = to_search.drop_duplicates()

In [21]:
to_search['track_name'] = to_search['track_name'].apply(lambda x: remove_space(x))

In [22]:
# save df to pickle
to_search.to_pickle("to_search.pkl")

In [2]:
# read df from pickle
to_search = pd.read_pickle("to_search.pkl")
to_search = to_search.reset_index().drop(['index'],axis=1)

In [3]:
to_search[to_search.track_name == 'No Sympathy']

,artist_name,track_name,popularity,genre
89533,Peter Tosh,No Sympathy,38,Reggae
91737,The Wailers,No Sympathy,31,Reggae
91882,Bob Marley & The Wailers,No Sympathy,29,Reggae
139051,The Wailers,No Sympathy,31,Ska


In [4]:
to_search = to_search[to_search.index >= 89533]

In [5]:
to_search

,artist_name,track_name,popularity,genre
89533,Peter Tosh,No Sympathy,38,Reggae
89534,Summer Thieves,Love Is Lost,47,Reggae
89535,Matisyahu,Close My Eyes,36,Reggae
89536,Slightly Stoopid,Drink Professionally,31,Reggae
89537,Wookiefoot,Loose Your Mind,33,Reggae
...,...,...,...,...
173883,Slave,Son Of Slide,39,Soul
173884,Jr Thomas & The Volcanos,Burning Fire,38,Soul
173885,Muddy Waters,,47,Soul
173886,R.LUM.R,With My Words,44,Soul


## Load musics from Genius

In [6]:
import requests
from bs4 import BeautifulSoup


def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + '6F7cdWVweWfmPXo-Nm-weqZ8_ouzvIk_ZfEXpNojIBdimY6ZI_e2zjPQiCcIU6YT'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html5lib')

    return html

Test by directly create different rows for different writers of the same song

In [7]:
# if already created
artists_writers_df = pd.read_pickle("artists_writers_df.pkl")
artists_writers_df = artists_writers_df.drop_duplicates(['track_name', 'artist'])

In [46]:
artists_writers_df = pd.DataFrame(columns=['track_name', 'artist', 'genre', 'popularity', 'more_infos'])

In [ ]:
counter = 0
for row in to_search.iterrows():
    artist = row[1]['artist_name']
    song = row[1]['track_name']
    popularity = row[1]['popularity']
    genre = row[1]['genre']
    
    r = request_song_info(song, artist)
    content = r.json()
    if len(content['response']['hits']) != 0:
        counter +=1
        true_content = content['response']['hits'][0]['result']

        url = true_content['url']
        soup2 = scrap_song_url(url)

        infos = soup2.find_all('meta')[-1]

        print(song + ', ' + artist + ', ' + str(popularity))
        to_add = {'track_name': song, 'artist': artist,\
                  'popularity': popularity, 'genre': genre, \
                  'more_infos':(str(infos).replace('&quot;', '').split('label:')[1:])}
        
        artists_writers_df = artists_writers_df.append(to_add, ignore_index=True)
        
        if counter%1000 ==0:
            # save df to pickle
            print('saving ...')
            artists_writers_df.to_pickle("artists_writers_df.pkl")
            print('saved')

No Sympathy, Peter Tosh, 38
Love Is Lost, Summer Thieves, 47
Close My Eyes, Matisyahu, 36
Drink Professionally, Slightly Stoopid, 31
Never Ever, SOJA, 42
Still the same, The Holdup, 31
Speak Easy, 311, 38
East Point, Rambutan Jam Band, 40
Ever Really Wanted, Slightly Stoopid, 31
Simplicity, Nahko and Medicine for the People, 31
Food For Thought, UB40, 49
Head In My Hands, The Expendables, 33
One More Night, One Drop, 33
My League, Mavado, 44
Visions , Dirty Heads, 31
The Truth, The Holdup, 31
After Another, Spawnbreezie, 40
My Lord, Michael Franti & Spearhead, 33
Planet Junior, Babe Rainbow, 45
Zion Gate Dub, King Tubby, 40
I Ain't Running, Sticky Fingers, 48
Move Like Creatures, Katastro, 33
Perfect, Christopher Martin, 46
Extension, 311, 39
Just Remember, House Of Shem, 43
A Little Bit Of Riddim, Michael Franti & Spearhead, 33
Time Won't Wait, Slightly Stoopid, 31
One True, Mishka, 39
Juicy Ones, Sticky Fingers, 48
Better, SOJA, 38
Expectations, Katastro, 33
No Way No , Los Pericos, 

Come Back Home, Beres Hammond, 35
In The Rain, Ernest Ranglin, 36
Natural Mystic, Bob Marley & The Wailers, 40
Everything You Need, Slightly Stoopid, 31
Like Your Style, Pepper, 30
Lincoln Highway Dub, Sublime, 33
Grass Is Greener, Bumpin Uglies, 31
Pressure, Skindred, 47
Lejos, Black Guayaba, 34
Garden Grove, Sublime, 36
Better People, Xavier Rudd, 45
Love's Contagious, Tarrus Riley, 37
Don't Stay Home, 311, 37
Sensemelia Persecution, Buju Banton, 34
Exodus, Bob Marley & The Wailers, 38
Black Cat, Ziggy Marley, 36
Rivers Of Babylon, Sublime, 31
Intro, Damian Marley, 40
Marijuana, Richie Spice, 39
Wings Of The Morning, Capleton, 32
Beat & Teach, Mavado, 39
Can't Get Me Down, Warrior King, 37
Falling For You, Tribal Theory, 32
Tune In, Gregory Isaacs, 35
Kinky Reggae, Bob Marley & The Wailers, 37
Somos Sur, Ana Tijoux, 52
Fight, Anthony B, 0
Only Woman D.J. With Degree, Sister Nancy, 36
More Than Dub , Rebelution, 30
Astuta, Santaferia, 60
My Love, Fortunate Youth, 32
Keep The Faith, An

Ring the Alarm , Buju Banton, 33
Big Salty Tears, The Ziggens, 32
Mal De Amore, Vicente Garcia, 45
She Wants to Destroy Me, Ballyhoo!, 31
Right Back, Sublime, 35
Por Cultivar Marihuana, Resistencia Suburbana, 52
Bocanada, Cultura Profética, 45
Jellyfish, Ocean Alley, 49
My World, Sublime With Rome, 31
Come & Go, Aer, 36
Stepping Out, Steel Pulse, 36
Love Black Woman, Buju Banton, 33
Hello Bonjour, Michael Franti & Spearhead, 32
Bending , Katastro, 31
Leisure, Israel Vibration, 36
Kingdom Of Zion, Alborosie, 43
Kindness For Weakness, Glen Washington, 38
Rolling Stone , Slightly Stoopid, 32
Soul Rebels, Bob Marley & The Wailers, 38
Fight This Feeling, Shaggy, 39
Can't Hardly Wait, Ballyhoo!, 31
Get Myself To Saturday, Michael Franti & Spearhead, 31
Music, The Beautiful Girls, 30
Music Move Me, The Expendables, 31
Safe And Sound, Sublime With Rome, 30
Lluvia, Nubes, Tristeza, Dread Mar I, 51
Still Standing, Michael Franti & Spearhead, 31
Shade from the Sun, Matisyahu, 35
The Better Half O

Am-A-Do, Bob Marley & The Wailers, 35
Running Back, Pacific Dub, 31
Unbreakable , Alborosie, 42
So True, The Black Seeds, 32
Bowl for Two , The Expendables, 32
Nothing I Wouldn't Do, The Expendables, 31
Dear Congress, , Dispatch, 36
Tomorrow's Another Day, Collie Buddz, 33
Square Circles , Matisyahu, 33
Invasion , Rebelution, 30
Do It For The Love, Michael Franti & Spearhead, 30
Exodus, Bob Marley & The Wailers, 38
Jahwakening, Mike Love, 37
I Got Love For Ya, Michael Franti & Spearhead, 31
Exaltation, Matisyahu, 34
So High, Arise Roots, 30
World Revolution, Ziggy Marley, 29
Love Is, Jah Cure, 38
Sweet Fanta Diallo, Alpha Blondy, 29
Burn Dem Down, Capleton, 43
Hello Africa, Garnett Silk, 35
Complaint, Buju Banton, 33
Core, MAGIC!, 41
Fine Day Together, SDIB, 31
Hot Cookin', G. Love & Special Sauce, 30
My Generation, Nas & Damian "Jr. Gong" Marley, 29
Shadow, SOJA, 33
Nuh Linga, Elephant Man, 28
Tear It Down, SOJA, 40
Engine Trouble, 10 Ft. Ganja Plant, 32
Dem Gone, Gentleman, 37
Just W

Taxi Driver, Steel Pulse, 34
New Love, Kash'd Out, 28
Talking Blues, The Gladiators, 38
Let Love In, Dennis Brown, 37
Shalom Salaam, Ziggy Marley, 35
Freedom Fighter, Anthony B, 46
Adelphia, The Cat Empire, 45
Undivided, Groundation, 38
For Real, Natural Incense, 29
Come Down, Collie Buddz, 34
Natty Dread A Weh She Want, Horace Andy, 37
What You Like, Pacific Dub, 30
Si No Te Tengo, Green Valley, 55
In With the Newport, Wheeland Brothers, 29
Awake, The Hip Abduction, 28
For The Love Of You, John Holt, 38
Do It Twice, The Wailers, 33
A Pocketful Of Shells, Mat McHugh, 42
Inspiration, E.N Young, 30
Personalmente, Las Pelotas, 64
Street Corner, The Skatalites, 46
Your Best Friend, Morgan Heritage, 36
Letter to Lady J, Dispatch, 34
Sun Is Shining, Bob Marley & The Wailers, 35
WHY MUST I CRY, Peter Tosh, 33
I Don't Wanna Live on Mars, Ziggy Marley, 28
Walk Tall, Ziggy Marley, 32
Let Her Dance, Stephen Marley, 31
Home, Iration, 29
Nada De Ti, Dread Mar I, 50
Pimp Floyd, Desmond Cheese, 40
Vi

...

I'm Back, Ky-Mani Marley, 35
Dark Angel, Tribal Seeds, 28
Nena, Dread Mar I, 48
Inspiracion, Dread Mar I, 49
Goodbe to My Haters, Mavado, 39
Where Is My Mind, The Supervillains, 32
Goodas Gal, Cham, 32
Give It Up , Pepper, 27
That Ghetto Girl, Richie Spice, 36
Positive, Fortunate Youth, 29
Dance & Shout, Shaggy, 33
Véngase Ii, Rawayana, 47
Resistencia indigena, Sig Ragga, 46
Feels Like Magic, Sweet & Irie, 46
New Roots , Iya Terra, 29
Sacar la Voz, Ana Tijoux, 53
Hypocrite, The Wailers, 32
Burnin' & Lootin', Bob Marley & The Wailers, 34
Jarmin' in the Dark, Young Monks, 35
Bal Shem Tov, Matisyahu, 26
Deixa o Menino Jogar, Natiruts, 55
Mini Bus, Barrington Levy, 34
Dope Remix Edit, Desmond Cheese, 42
Faster, The Holdup, 26
Symbol Of Reality, The Gladiators, 37
Splintered Heart, Iration, 27
Irie Insulation, Inna Vision, 28
Nothing Else Quite Like Home, G. Love & Special Sauce, 28
Legalize It, Sublime, 29
Ska Diddy , Slightly Stoopid, 28
Babylon Makes The Rules, Steel Pulse, 25
Imposible,

Garbage Grove, Sublime, 29
IDK, Lyall Moloney, 32
Volverte a Ver , Fyahbwoy, 43
Born & Razed, Dispatch, 31
Open Up The Gate, The Congos, 34
Worlds on Fire, Skindred, 34
Suena la Alarma, Los Cafres, 46
Gracias Vida, Cultura Profética, 40
Some Girls Are Crazy, Echo Movement, 27
Blues Away, Groundation, 36
Light It Up, The Hip Abduction, 26
End, Bumpin Uglies, 26
Glad To Be Home, The Holdup, 26
Hive, 311, 33
Please Me, Glen Washington, 29
High & Windy, Sara Lugo, 41
Desdemona, Hollie Cook, 26
Heathen, Bob Marley & The Wailers, 35
El Silencio, Los Cafres, 46
Hardcore , Lady Saw, 28
One Drop , The Expendables, 28
Positive Vibration, Bob Marley & The Wailers, 32
Bridges, Shaggy, 38
It's a New Day, Aer, 30
Make Me Yours, Phyllis Dillon, 31
Poor Millionaire, Gregory Isaacs, 30
Fought for Dis, Sizzla, 32
Collie Weed, Barrington Levy, 33
The Big Tree, Stand High Patrol, 53
Weh Daddy Deh, Vybz Kartel, 37
Foolish Fool, Sublime, 30
Move You Body, Buju Banton, 30
Fill Us Up With Your Mercy, Garnett 

Everybody Needs Love, Slim Smith, 37
Volero, Rawayana, 44
Under My Tongue, Katastro, 26
Overdrive, Simplified, 26
Life on Earth, Lord Echo, 38
Runway, Snow, 35
At Last, Wookiefoot, 27
Summertime, Vybz Kartel, 40
Unruly Law, Popcaan, 31
Wanna Know, Groundation, 25
Tie Yuh, Mavado, 39
Roots, Rock, Reggae, Bob Marley & The Wailers, 31
Crisis, Bob Marley & The Wailers, 32
Where You Gonna Run, Peter Tosh, 34
I'm the Toughest, Peter Tosh, 34
Mama She Don't Like You , Alborosie, 41
Stop That Train, Peter Tosh, 32
Concrete, Bob Marley & The Wailers, 32
Lonely Lover, Shaggy, 30
Plan, Movimiento Original, 54
She Loves Me Now, Cocoa Tea, 32
Great Train Robbery, Black Uhuru, 34
Come Again, Cocoa Tea, 32
Un Accidente, Circo, 29
Mystery Angel, Xavier Rudd, 41
Resin, The Supervillains, 30
Utterance, Black Uhuru, 29
John Crow, Jimmy Cliff, 30
Love and Unity, Michael Prophet, 35
Wicked Dickie , Buju Banton, 31
Look Girl, Barrington Levy, 30
Hear My Cry Oh Lord, Marvia Providence, 34
Loud And Clear, Ski

Glamour Tropical , Natiruts, 53
Jah Blessing , Sizzla, 29
White Moth, Xavier Rudd, 37
Minha Felicidade, Gabriel Elias, 61
#ТАМАДА, MiyaGi, 42
Espero Que um Dia, Natiruts, 53
Shelter Of Your Love, Jimmy Cliff, 24
To the Sky, Pacific Dub, 27
Without Love, The Expendables, 26
Hello Sunshine, Jimmy Cliff, 24
Pass Me the Sensi, Stick Figure, 26
Rules of the Dance, Mungo's Hi Fi, 47
Piece Of My Heart, Shaggy, 38
Dance Monkey, MAGIC!, 37
Inna Jamaica, Mellow Mood, 46
Vibration Dub, Fortunate Youth, 28
Collie Man, Slightly Stoopid, 27
Love It, Vybz Kartel, 32
You Want Me, Vybz Kartel, 31
Mr. Brown, Bob Marley & The Wailers, 30
One Love / People Get Ready, Bob Marley & The Wailers, 25
Baby, UB40, 23
Hey Baby, Stephen Marley, 27
Goggle, Tanya Stephens, 30
Añejo, Rawayana, 43
Koo Koo, Toots & The Maytals, 31
Flor del Potrero, Los Cafres, 41
Revolution, Toots & The Maytals, 33
Feel the Skank, Lutan Fyah, 32
Ese Loco, Slightly Stoopid, 25
War & Crime, Lucky Dube, 34
Make Me Happy, Alton Ellis, 33
D

Natural Mystic [feat. Ky-Mani Marley], Alborosie, 39
Three Blind Mice, Big Youth, 0
Give Me, HIRIE, 26
Trinity, Tomorrows Bad Seeds, 27
Welcome, 311, 29
Memories, Beenie Man, 29
In the Right Mood, 77 Jefferson, 26
Bootleggers, Ballyhoo!, 26
Golden Hen, Tenor Saw, 33
Always on My Mind, Tiki Taane, 53
Si Tú Me Quieres, Gomba Jahbari, 31
Knowledge, Bushman, 0
Bad Up Who, Busy Signal, 31
Sweet Lies, Beres Hammond, 28
Do The Reggay, Toots & The Maytals, 32
Rat-at-at, The Skints, 37
This World, Bushman, 0
Grave Yard, Tarrus Riley, 33
Start A New, Tarrus Riley, 32
Up Against the Wall Riddim, The Skints, 38
Scarlet Begonias, Sublime, 29
Hola, The Movement, 24
Easy Like Sunday Morning, Christopher Martin, 33
Sound System, Steel Pulse, 23
Craven Choke Puppy, The Wailers, 28
Struggling, Bumpin Uglies, 24
Orgullo, Moral Distraida, 58
Watch The Walls Melt Down, Matisyahu, 24
Get Back , Bumpin Uglies, 25
Breathe Easy, Matisyahu, 24
Fantasy, MiyaGi, 41
Live Your Life, Bumpin Uglies, 24
Joy In The Mor

In [15]:
# save df to pickle
artists_writers_df.to_pickle("artists_writers_df.pkl")

In [52]:
artists_writers_df = artists_writers_df.reset_index().drop(['index'], axis=1)

In [11]:
artists_writers_df

,track_name,artist,genre,popularity,more_infos
0,Be Without You,Mary J. Blige,R&B,65,"[Strings,artists:[{_type:artist,api_path:/arti..."
1,Desperado,Rihanna,R&B,63,"[Distributor,artists:[{_type:artist,api_path:/..."
2,Ice On My Baby,Yung Bleu,R&B,62,"[Video,overlay_compatible_image_url:https://im..."
3,Heaven Falls / Fall on Me,Surfaces,R&B,61,[]
4,Love Myself,Olivia O'Brien,R&B,68,"[Video,overlay_compatible_image_url:https://im..."
...,...,...,...,...,...
70006,Si el Amor Se Cae,Los Cafres,Reggae,54,[]
70007,Les copains d'abord,Georges Brassens,Reggae,36,[]
70008,Rice Water,Dispatch,Reggae,37,[]
70009,Visions,Dirty Heads,Reggae,33,[]
